# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import copy
import datetime
from rentcast_dapi import *

%load_ext autoreload
%autoreload 2

# RentCast Overview

In [ ]:
# API docs - https://developers.rentcast.io/reference/introduction
# Pricing - https://www.rentcast.io/api#api-pricing
# Account Dashboard - https://app.rentcast.io/app/api

rent_cast_api_key = 'fcc15170bef044e896b098360c6fe570'

# Grab Data - Single Address

In [ ]:
address = '1454 Jefferson Ave, Redwood City, CA, 94062'

In [ ]:
url = 'https://api.rentcast.io/v1/properties'
querystring = {'address': address}
headers = {'accept': 'application/json', 'X-Api-Key': rent_cast_api_key}

response = requests.get(url, headers=headers, params=querystring)
response.json()

# Use simple API wrapper

In [ ]:
rcd = RentCastData(city='Atherton', offset_lim=1, api_key=rent_cast_api_key)
rcd.data_processed

# Grab Data - Lots of Listings per City

In [ ]:
# Using Rentcast API

city = 'Redwood City'
city = 'Atherton'
city = 'Portland'
offsets = np.arange(0, 10, 1)
df_all = None

for offset in offsets:

    # This is the max limit per query
    # Offsets allow for grabbing more data, but will count as an additional query
    limit = 500  

    # Design query
    url = 'https://api.rentcast.io/v1/properties'
    querystring = {'city': city, 'state': 'OR', 'limit': limit, 'offset': offset * limit}
    headers = {'accept': 'application/json', 'X-Api-Key': rent_cast_api_key}

    # Get query
    response = requests.get(url, headers=headers, params=querystring)
    listings = response.json()



    # Loop through results and create dataframe
    for listing_loop in listings:

        listing = copy.deepcopy(listing_loop)
        listing_temp = copy.deepcopy(listing)

        headers = ['id', 'addressLine1', 'addressLine2', 'city', 'state', 
                   'zipCode', 'county', 'latitude', 'longitude', 'squareFootage', 
                   'lotSize', 'yearBuilt', 'lastSaleDate', 'lastSalePrice', 
                   'bedrooms', 'bathrooms', 'propertyType']

        for header in listing_temp:
            if header not in headers:
                del listing[header]
            else:
                listing[header] = [listing[header]]

        df_temp = pd.DataFrame.from_dict(listing)
        df_all = pd.concat([df_all, df_temp], ignore_index=True)

# Save to CSV file

In [ ]:
city = 'Redwood City'
filename_city = city.replace(' ', '_').lower()
df_all.to_csv('%s.csv' % filename_city)

# Simple Plots

In [ ]:
check1 = df['squareFootage'] > 1000
check2 = df['squareFootage'] < 2000
df_temp  = df[check1 & check2]
# df_temp = df

x = df_temp['months'].values / 12
y = df_temp['price_per_sqft'].values

# Avererages
grouped_df = df_temp.groupby('months')['price_per_sqft'].mean().reset_index()
x_avg = grouped_df['months'].values / 12
y_avg = grouped_df['price_per_sqft'].values

# y_average = df.average
plt.figure(figsize=(10, 4))

# plt.plot(x, y, lw=0, ms=2, marker='o', alpha=0.2)
# plt.plot(x_avg, y_avg, lw=1, ms=0, marker='o', color='black', alpha=0.5)
# plt.ylim([-10, 2000])

plt.semilogy(x, y, lw=0, ms=2, marker='o', alpha=0.2)
plt.semilogy(x_avg, y_avg, lw=1, ms=0, marker='o', color='black', alpha=0.5)
plt.ylim([50, 2000])


# plt.xlim([40, 60])

# x labels
int_labels = np.arange(0, 60, 2.5)
n = len(int_labels)
lsd = df['lastSaleDate'].values[0]
dt = datetime.datetime.strptime(lsd, '%Y-%m-%dT%H:%M:%S.%fZ')
dt_labels = [add_months(dt,x * 30) for x in range(n)]
dt_labels = [dt_label.strftime('%m/%Y') for dt_label in dt_labels]
plt.xticks(int_labels, dt_labels, rotation=45)

plt.grid(True)